Notebook 02: Exploratory Data Analysis (EDA)

# 02 – Exploratory Data Analysis (EDA)

## Sistema Inteligente de Retenção de Clientes

**Autor:** Ivan  
**Dataset:** Telco Customer Churn (limpo no Notebook 01)  
**Objetivo:** Explorar padrões, relações e insights nos dados para embasar a modelagem

---

### Estrutura do Notebook

1. Análise Univariada (distribuições individuais)
2. Análise da Variável Alvo (Churn)
3. Análise Bivariada (relações com Churn)
4. Análise de Correlações
5. Segmentação de Clientes
6. Insights de Negócio
7. Conclusões e Recomendações

## 1. Configuração Inicial

In [ ]:
#Importado as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

#Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("Bibliotecas carregadas com sucesso!")

## 2. Carregamento dos Dados Limpos

Carregando o dataset processado no Notebook 01.

In [ ]:
# Carregar dados limpos
'''PROCESSED_DATA_PATH = Path("../data/processed/churn_data_clean.csv")
df = pd.read_csv(PROCESSED_DATA_PATH)
'''

df = pd.read_csv('processed_data.csv')

print("="*80)
print("DATASET CARREGADO")
print("="*80)
print(f"\nDimensões: {df.shape[0]:,} linhas × {df.shape[1]} colunas")
print(f"\nPrimeiras linhas:\n")
display(df.head())

## 3. Visão Geral dos Dados

In [ ]:
print("\nINFORMAÇÕES GERAIS:\n")
print(df.info())

In [ ]:
print("\nESTATÍSTICAS DESCRITIVAS - NUMÉRICAS:\n")
display(df.describe())

In [ ]:
print("\nESTATÍSTICAS DESCRITIVAS - CATEGÓRICAS:\n")
display(df.describe(include='object'))

## 4. Análise Univariada

Explorando a distribuição de cada variável individualmente.

### 4.1 Variáveis Demográficas

#### 4.1.1. Distribuição de Churn

In [ ]:
#Analisando a variável target

print("DISTRIBUIÇÃO DO CHURN:")
churn_counts = df['Churn'].value_counts()
churn_percentage = df['Churn'].value_counts(normalize=True) * 100

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Contagem de Churn', 'Percentual de Churn'),
    specs=[[{'type':'bar'}, {'type':'pie'}]]
)

fig.add_trace(
    go.Bar(x=churn_counts.index, y=churn_counts.values,
           text=churn_counts.values, textposition='auto',
           marker_color=['#2ecc71', '#e74c3c']),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=churn_counts.index, values=churn_counts.values,
           hole=.3, marker_colors=['#2ecc71', '#e74c3c']),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False,
                  title_text="Distribuição da Variável Target (Churn)")
fig.show()

print(f"Total de clientes: {len(df)}")
print(f"Clientes que permaneceram (No): {churn_counts['No']} ({churn_percentage['No']:.1f}%)")
print(f"Clientes que cancelaram (Yes): {churn_counts['Yes']} ({churn_percentage['Yes']:.1f}%)")
print("\n" + "="*50)

#### 4.1.2. Distribuições (Genero, Idade, Parceiro, Dependentes)

In [ ]:
#Criar subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

#Gender
gender_counts = df['gender'].value_counts()
axes[0, 0].pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%',
               colors=['#3498db', '#e74c3c'], startangle=90)
axes[0, 0].set_title('Distribuição por Gênero', fontweight='bold', fontsize=12)

#SeniorCitizen
senior_counts = df['SeniorCitizen'].value_counts()
sns.barplot(x=senior_counts.index, y=senior_counts.values, ax=axes[0, 1], palette='viridis')
axes[0, 1].set_title('Distribuição por Idade (Senior Citizen)', fontweight='bold', fontsize=12)
axes[0, 1].set_xlabel('Senior Citizen (0=Não, 1=Sim)')
axes[0, 1].set_ylabel('Quantidade')

#Partner
partner_counts = df['Partner'].value_counts()
sns.barplot(x=partner_counts.index, y=partner_counts.values, ax=axes[1, 0], palette='muted')
axes[1, 0].set_title('Distribuição - Possui Parceiro', fontweight='bold', fontsize=12)
axes[1, 0].set_xlabel('Possui Parceiro')
axes[1, 0].set_ylabel('Quantidade')

#Dependents
dep_counts = df['Dependents'].value_counts()
sns.barplot(x=dep_counts.index, y=dep_counts.values, ax=axes[1, 1], palette='Set2')
axes[1, 1].set_title('Distribuição - Possui Dependentes', fontweight='bold', fontsize=12)
axes[1, 1].set_xlabel('Possui Dependentes')
axes[1, 1].set_ylabel('Quantidade')

plt.tight_layout()
plt.show()

#### 4.1.3. Churn por (Genero, Idade, Parceiro, Dependentes)

In [ ]:
# Análise demográfica dos clientes

demographic_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
fig = plt.figure(figsize=(15, 10))

for i, col in enumerate(demographic_cols, 1):
    plt.subplot(2, 2, i)

    # Calcular percentuais
    temp_df = df.groupby([col, 'Churn']).size().unstack()
    temp_percent = temp_df.div(temp_df.sum(axis=1), axis=0) * 100

    # Plot
    temp_percent.plot(kind='bar', stacked=True,
                      color=['#2ecc71', '#e74c3c'], ax=plt.gca())
    plt.title(f'Churn por {col}', fontsize=14, fontweight='bold')
    plt.xlabel(col)
    plt.ylabel('Percentual (%)')
    plt.legend(title='Churn', labels=['Não', 'Sim'])
    plt.xticks(rotation=0)

    # Adicionar valores nas barras
    for p in plt.gca().patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy()
        if height > 0:
            plt.gca().annotate(f'{height:.1f}%',
                             (x + width/2, y + height/2),
                             ha='center', va='center',
                             fontsize=9, color='white')

plt.tight_layout()
plt.show()

### 4.2 Variáveis de Serviços

#### 4.2.1. Analise dos serviços contratados

In [ ]:
# Serviços principais
service_cols = ['PhoneService', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, col in enumerate(service_cols):
    counts = df[col].value_counts()
    sns.barplot(x=counts.index, y=counts.values, ax=axes[idx], palette='coolwarm')
    axes[idx].set_title(col, fontweight='bold', fontsize=11)
    axes[idx].set_xlabel('')
    axes[idx].set_ylabel('Quantidade')
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

#### 4.2.2. Churn por serviços contratados

In [ ]:
service_cols = ['PhoneService', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
# Flatten the axes array for easier iteration
axes = axes.flatten()

for i, col in enumerate(service_cols):
    ax = axes[i]

    # Calcular percentuais
    temp_df = df.groupby([col, 'Churn']).size().unstack()
    temp_percent = temp_df.div(temp_df.sum(axis=1), axis=0) * 100

    # Plot
    temp_percent.plot(kind='bar', stacked=True,
                      color=['#2ecc71', '#e74c3c'], ax=ax)
    ax.set_title(f'Churn por {col}', fontsize=14, fontweight='bold')
    ax.set_xlabel(col)
    ax.set_ylabel('Percentual (%)')
    ax.legend(title='Churn', labels=['Não', 'Sim'])
    ax.tick_params(axis='x', rotation=0)

    # Adicionar valores nas barras
    for p in ax.patches:
        width, height = p.get_width(), p.get_height()
        x, y = p.get_xy()
        if height > 0:
            ax.annotate(f'{height:.1f}%',
                             (x + width/2, y + height/2),
                             ha='center', va='center',
                             fontsize=9, color='white')

plt.tight_layout()
plt.show()

#### 4.2.3. Taxa de Serviços Contratados

In [ ]:
# Estatísticas de adoção
print("\nTaxa de Adoção de Serviços:\n")

for col in service_cols:
    yes_pct = (df[col] == 'Yes').mean() * 100
    print(f"  • {col}: {yes_pct:.1f}%")

### 4.3 Variáveis Financeiras

#### 4.3.1 Analise Financeira (Distribuição por Serviçooo)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Tenure
axes[0].hist(df['tenure'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Distribuição - Tempo de Contrato (Tenure)', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Meses')
axes[0].set_ylabel('Frequência')
axes[0].axvline(df['tenure'].mean(), color='red', linestyle='--', label=f'Média: {df["tenure"].mean():.1f}')
axes[0].legend()

# MonthlyCharges
axes[1].hist(df['MonthlyCharges'], bins=30, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].set_title('Distribuição - Cobrança Mensal', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Valor (R$)')
axes[1].set_ylabel('Frequência')
axes[1].axvline(df['MonthlyCharges'].mean(), color='red', linestyle='--',
                label=f'Média: R$ {df["MonthlyCharges"].mean():.2f}')
axes[1].legend()

# TotalCharges
axes[2].hist(df['TotalCharges'], bins=30, color='lightgreen', edgecolor='black', alpha=0.7)
axes[2].set_title('Distribuição - Cobrança Total', fontweight='bold', fontsize=12)
axes[2].set_xlabel('Valor (R$)')
axes[2].set_ylabel('Frequência')
axes[2].axvline(df['TotalCharges'].mean(), color='red', linestyle='--',
                label=f'Média: R$ {df["TotalCharges"].mean():.2f}')
axes[2].legend()

plt.tight_layout()
plt.show()


#### 4.3.2 Distribuição de Churn por Serviço)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Tenure by Churn
sns.histplot(data=df, x='tenure', hue='Churn', kde=True, ax=axes[0], palette=['#2ecc71', '#e74c3c'])
axes[0].set_title('Distribuição de Tempo de Contrato (Tenure) por Churn', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Meses de Contrato')
axes[0].set_ylabel('Frequência')

# MonthlyCharges by Churn
sns.histplot(data=df, x='MonthlyCharges', hue='Churn', kde=True, ax=axes[1], palette=['#2ecc71', '#e74c3c'])
axes[1].set_title('Distribuição de Cobrança Mensal por Churn', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Valor Mensal (R$)')
axes[1].set_ylabel('Frequência')

# TotalCharges by Churn
sns.histplot(data=df, x='TotalCharges', hue='Churn', kde=True, ax=axes[2], palette=['#2ecc71', '#e74c3c'])
axes[2].set_title('Distribuição de Cobrança Total por Churn', fontweight='bold', fontsize=12)
axes[2].set_xlabel('Valor Total (R$)')
axes[2].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

#### 4.3.3 Estatísticas financeiras

In [ ]:
# Estatísticas financeiras
print(f"  • Tenure médio: {df['tenure'].mean():.1f} meses")
print(f"  • Cobrança mensal média: {df['MonthlyCharges'].mean():.2f}")
print(f"  • Cobrança total média: {df['TotalCharges'].mean():.2f}")
print(f"  • Receita total: {df['MonthlyCharges'].sum():,.2f}/mês")

### 4.4 Tipo de Contrato e Pagamento

#### 4.4.1 Visualizando o Tipo de Contrato e Pagamento

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Contract
contract_counts = df['Contract'].value_counts()
axes[0].pie(contract_counts, labels=contract_counts.index, autopct='%1.1f%%',
            colors=['#FF6B6B', '#4ECDC4', '#45B7D1'], startangle=90)
axes[0].set_title('Distribuição por Tipo de Contrato', fontweight='bold', fontsize=12)

# PaperlessBilling
billing_counts = df['PaperlessBilling'].value_counts()
sns.barplot(x=billing_counts.index, y=billing_counts.values, ax=axes[1], palette='Set3')
axes[1].set_title('Faturamento Sem Papel', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Paperless Billing')
axes[1].set_ylabel('Quantidade')

# PaymentMethod
payment_counts = df['PaymentMethod'].value_counts()
sns.barplot(y=payment_counts.index, x=payment_counts.values, ax=axes[2], palette='pastel')
axes[2].set_title('Métodos de Pagamento', fontweight='bold', fontsize=12)
axes[2].set_xlabel('Quantidade')
axes[2].set_ylabel('')

plt.tight_layout()
plt.show()

#### 4.4.2 Visualizando o Churn por Tipo de Contrato e Pagamento

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Contract by Churn
temp_df_contract = df.groupby(['Contract', 'Churn']).size().unstack()
temp_percent_contract = temp_df_contract.div(temp_df_contract.sum(axis=1), axis=0) * 100
temp_percent_contract.plot(kind='bar', stacked=True, color=['#2ecc71', '#e74c3c'], ax=axes[0])
axes[0].set_title('Churn por Tipo de Contrato', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Tipo de Contrato')
axes[0].set_ylabel('Percentual (%)')
axes[0].legend(title='Churn', labels=['Não', 'Sim'])
axes[0].tick_params(axis='x', rotation=0)
for p in axes[0].patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy()
    if height > 0:
        axes[0].annotate(f'{height:.1f}%', (x + width/2, y + height/2), ha='center', va='center', fontsize=9, color='white')

# PaperlessBilling by Churn
temp_df_billing = df.groupby(['PaperlessBilling', 'Churn']).size().unstack()
temp_percent_billing = temp_df_billing.div(temp_df_billing.sum(axis=1), axis=0) * 100
temp_percent_billing.plot(kind='bar', stacked=True, color=['#2ecc71', '#e74c3c'], ax=axes[1])
axes[1].set_title('Churn por Faturamento Sem Papel', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Paperless Billing')
axes[1].set_ylabel('Percentual (%)')
axes[1].legend(title='Churn', labels=['Não', 'Sim'])
axes[1].tick_params(axis='x', rotation=0)
for p in axes[1].patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy()
    if height > 0:
        axes[1].annotate(f'{height:.1f}%', (x + width/2, y + height/2), ha='center', va='center', fontsize=9, color='white')

# PaymentMethod by Churn
temp_df_payment = df.groupby(['PaymentMethod', 'Churn']).size().unstack()
temp_percent_payment = temp_df_payment.div(temp_df_payment.sum(axis=1), axis=0) * 100
temp_percent_payment.plot(kind='bar', stacked=True, color=['#2ecc71', '#e74c3c'], ax=axes[2])
axes[2].set_title('Churn por Método de Pagamento', fontweight='bold', fontsize=12)
axes[2].set_xlabel('Método de Pagamento')
axes[2].set_ylabel('Percentual (%)')
axes[2].legend(title='Churn', labels=['Não', 'Sim'])
axes[2].tick_params(axis='x', rotation=45)
for p in axes[2].patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy()
    if height > 0:
        axes[2].annotate(f'{height:.1f}%', (x + width/2, y + height/2), ha='center', va='center', fontsize=9, color='white')

plt.tight_layout()
plt.show()

#### 4.4.3 Resumo de Contratos e Pagamentos

In [ ]:
print('--- Resumo de Contratos ---')
display(contract_counts.to_frame(name='Count'))

In [ ]:
print('\n--- Resumo de Faturamento Sem Papel ---')
display(billing_counts.to_frame(name='Count'))

In [ ]:
print('\n--- Resumo de Métodos de Pagamento ---')
display(payment_counts.to_frame(name='Count'))

## 5. Análise da Variável Alvo (Churn)

Esta é a variável mais importante do nosso projeto.

#### 5.1 Distribuição de Churn

In [ ]:
churn_counts = df['Churn'].value_counts()
churn_pct = df['Churn'].value_counts(normalize=True) * 100

print(f"\nDistribuição de Churn:\n")
print(f"  • Não Churn (No): {churn_counts.get('No', 0):,} ({churn_pct.get('No', 0):.1f}%)")
print(f"  • Churn (Yes): {churn_counts.get('Yes', 0):,} ({churn_pct.get('Yes', 0):.1f}%)")

#### 5.2 Visualização do Churn

In [ ]:
# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
sns.countplot(x='Churn', data=df, ax=axes[0], palette=['#2ecc71', '#e74c3c'])
axes[0].set_title('Contagem de Churn', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Quantidade')
axes[0].set_xlabel('Churn')

# Adicionar valores nas barras
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%d')

# Gráfico de pizza
colors = ['#2ecc71', '#e74c3c']
axes[1].pie(churn_counts, labels=['Não Churn', 'Churn'], autopct='%1.1f%%',
            colors=colors, startangle=90, explode=(0, 0.1))
axes[1].set_title('Proporção de Churn', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

## Insights Iniciais

**Observação:** O dataset está **desbalanceado**, com aproximadamente 73% de clientes que não fizeram churn e 27% que fizeram.

**Implicação:** Na modelagem, será necessário o uso de técnicas para lidar com desbalanceamento:

- Class weights
- SMOTE
- Métricas adequadas (AUC-ROC, F1-Score)


## 6. Análise Bivariada - Relação com Churn

Explorar como cada variável se relaciona com o Churn.

### 6.1 Churn vs Variáveis Demográficas

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Gender vs Churn
pd.crosstab(df['gender'], df['Churn'], normalize='index').plot(
    kind='bar', ax=axes[0, 0], color=['#2ecc71', '#e74c3c'], rot=0
)
axes[0, 0].set_title('Churn por Gênero', fontweight='bold', fontsize=12)
axes[0, 0].set_ylabel('Proporção')
axes[0, 0].set_xlabel('Gênero')
axes[0, 0].legend(['Não Churn', 'Churn'])

# SeniorCitizen vs Churn
pd.crosstab(df['SeniorCitizen'], df['Churn'], normalize='index').plot(
    kind='bar', ax=axes[0, 1], color=['#2ecc71', '#e74c3c'], rot=0
)
axes[0, 1].set_title('Churn por Idade (Senior Citizen)', fontweight='bold', fontsize=12)
axes[0, 1].set_ylabel('Proporção')
axes[0, 1].set_xlabel('Senior Citizen (0=Não, 1=Sim)')
axes[0, 1].legend(['Não Churn', 'Churn'])

# Partner vs Churn
pd.crosstab(df['Partner'], df['Churn'], normalize='index').plot(
    kind='bar', ax=axes[1, 0], color=['#2ecc71', '#e74c3c'], rot=0
)
axes[1, 0].set_title('Churn - Possui Parceiro', fontweight='bold', fontsize=12)
axes[1, 0].set_ylabel('Proporção')
axes[1, 0].set_xlabel('Possui Parceiro')
axes[1, 0].legend(['Não Churn', 'Churn'])

# Dependents vs Churn
pd.crosstab(df['Dependents'], df['Churn'], normalize='index').plot(
    kind='bar', ax=axes[1, 1], color=['#2ecc71', '#e74c3c'], rot=0
)
axes[1, 1].set_title('Churn - Possui Dependentes', fontweight='bold', fontsize=12)
axes[1, 1].set_ylabel('Proporção')
axes[1, 1].set_xlabel('Possui Dependentes')
axes[1, 1].legend(['Não Churn', 'Churn'])

plt.tight_layout()
plt.show()

### 6.2 Estatisticas - Churn vs Variáveis Demográficas

In [ ]:
# Estatísticas
print("\nTaxa de Churn por Categoria:\n")
for col in ['gender', 'SeniorCitizen', 'Partner', 'Dependents']:
    print(f"\n{col}:")
    churn_by_cat = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
    print(churn_by_cat.round(1))

### Insights Demográficos

1. **Gênero:** Não há diferença significativa na taxa de churn entre homens e mulheres

2. **Senior Citizens:** Clientes idosos têm **taxa de churn maior** (~41% vs ~23%)

3. **Parceiro:** Clientes **sem parceiro** têm taxa de churn maior (~33% vs ~20%)

4. **Dependentes:** Clientes **sem dependentes** têm taxa de churn maior (~31% vs ~15%)

**Conclusão:** Clientes idosos, solteiros e sem dependentes são **grupo de risco**.

### 6.3 Churn vs Tipo de Contrato

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Contract vs Churn (proporção)
contract_churn = pd.crosstab(df['Contract'], df['Churn'], normalize='index') * 100
contract_churn.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'], rot=45)
axes[0].set_title('Taxa de Churn por Tipo de Contrato', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Percentual (%)')
axes[0].set_xlabel('Tipo de Contrato')
axes[0].legend(['Não Churn', 'Churn'])

# Adicionar valores nas barras
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%.1f%%')

# Contract vs Churn (contagem absoluta)
pd.crosstab(df['Contract'], df['Churn']).plot(kind='bar', ax=axes[1],
                                                color=['#2ecc71', '#e74c3c'], rot=45)
axes[1].set_title('Quantidade de Clientes por Tipo de Contrato', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Quantidade')
axes[1].set_xlabel('Tipo de Contrato')
axes[1].legend(['Não Churn', 'Churn'])

plt.tight_layout()
plt.show()

In [ ]:
print("\n📊 Taxa de Churn por Tipo de Contrato:")
display(contract_churn.round(1))
display(pd.crosstab(df['Contract'], df['Churn']))

### Insight Crítico - Tipo de Contrato

**Descoberta mais importante até agora:**

 - **Month-to-month:** ~42% de churn (ALTO RISCO)
 - **One year:** ~11% de churn (MÉDIO RISCO)
 - **Two year:** ~3% de churn (BAIXO RISCO)

 **Ação recomendada:** Incentivar migração de contratos mensais para anuais/bianuais pode reduzir drasticamente o churn.


### 6.3 Churn vs Serviços de Internet

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

internet_services = ['InternetService', 'OnlineSecurity', 'OnlineBackup',
                     'DeviceProtection', 'TechSupport', 'StreamingTV']

for idx, col in enumerate(internet_services):
    service_churn = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
    service_churn.plot(kind='bar', ax=axes[idx], color=['#2ecc71', '#e74c3c'], rot=45)
    axes[idx].set_title(f'Churn - {col}', fontweight='bold', fontsize=11)
    axes[idx].set_ylabel('Percentual (%)')
    axes[idx].set_xlabel('')
    axes[idx].legend(['Não Churn', 'Churn'], loc='upper right')

plt.tight_layout()
plt.show()

### 6.4 Taxa de Churn por Serviço de Internet

In [ ]:
for col in internet_services:
    #print(f"\n{col}:")
    churn_by_service = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
    display(churn_by_service.round(1))

### Insights - Serviços de Internet

1. **Fiber optic:** Taxa de churn **muito alta** (42%) comparada a DSL (19%)

2. **Serviços de segurança:** Clientes **sem** OnlineSecurity, OnlineBackup, DeviceProtection têm churn maior

3. **TechSupport:** Clientes **sem** suporte técnico têm churn significativamente maior

**Hipótese:** Fiber optic pode ter problemas de qualidade ou preço alto sem valor percebido.

**Ação:** Investigar satisfação com Fiber optic e oferecer pacotes de segurança.

### 6.5 Churn vs Variáveis Financeiras

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Tenure vs Churn
sns.boxplot(x='Churn', y='tenure', data=df, ax=axes[0], palette=['#2ecc71', '#e74c3c'])
axes[0].set_title('Tenure vs Churn', fontweight='bold', fontsize=12)
axes[0].set_ylabel('Meses de Contrato')
axes[0].set_xlabel('Churn')

# MonthlyCharges vs Churn
sns.boxplot(x='Churn', y='MonthlyCharges', data=df, ax=axes[1], palette=['#2ecc71', '#e74c3c'])
axes[1].set_title('Cobrança Mensal vs Churn', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Valor Mensal (R$)')
axes[1].set_xlabel('Churn')

# TotalCharges vs Churn
sns.boxplot(x='Churn', y='TotalCharges', data=df, ax=axes[2], palette=['#2ecc71', '#e74c3c'])
axes[2].set_title('Cobrança Total vs Churn', fontweight='bold', fontsize=12)
axes[2].set_ylabel('Valor Total (R$)')
axes[2].set_xlabel('Churn')

plt.tight_layout()
plt.show()

### 6.6 Estatisticas - Churn vs Variáveis Financeiras

In [ ]:
# Estatísticas
print("\nComparação Financeira - Churn vs Não Churn:")
financial_comparison = df.groupby('Churn')[['tenure', 'MonthlyCharges', 'TotalCharges']].agg(['mean', 'median'])
display(financial_comparison.round(2))

### Insights Financeiros

1. **Tenure:** Clientes que fazem churn têm **tenure muito menor** (média 18 meses vs38 meses)

2. **MonthlyCharges:** Clientes que fazem churn pagam **mais por mês** (74 vs 61)

3. **TotalCharges:** Clientes que fazem churn têm **gasto total menor** (devido ao menor tenure)


**Conclusão:** Clientes novos com cobranças altas são **grupo de alto risco**.

**Ação:** Focar retenção nos primeiros 12 meses com ofertas especiais.

### 6.5 Churn vs Método de Pagamento

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# PaymentMethod vs Churn (proporção)
payment_churn = pd.crosstab(df['PaymentMethod'], df['Churn'], normalize='index') * 100
payment_churn.plot(kind='barh', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Taxa de Churn por Método de Pagamento', fontweight='bold', fontsize=14)
axes[0].set_xlabel('Percentual (%)')
axes[0].set_ylabel('Método de Pagamento')
axes[0].legend(['Não Churn', 'Churn'])

# PaperlessBilling vs Churn
billing_churn = pd.crosstab(df['PaperlessBilling'], df['Churn'], normalize='index') * 100
billing_churn.plot(kind='bar', ax=axes[1], color=['#2ecc71', '#e74c3c'], rot=0)
axes[1].set_title('Taxa de Churn - Paperless Billing', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Percentual (%)')
axes[1].set_xlabel('Paperless Billing')
axes[1].legend(['Não Churn', 'Churn'])

plt.tight_layout()
plt.show()

print("\nTaxa de Churn por Método de Pagamento:")
print(payment_churn.round(1))

print("\nTaxa de Churn - Paperless Billing:")
print(billing_churn.round(1))

### Insights - Pagamento

1. **Electronic check:** Taxa de churn **muito alta** (~45%)

2. **Outros métodos:** Churn entre 15-18% (muito menor)

3. **Paperless Billing:** Clientes com fatura digital têm churn ligeiramente maior


**Hipótese:** Electronic check pode indicar menor engajamento ou problemas de pagamento.

**Ação:** Incentivar migração para débito automático com desconto.

## 7. Análise de Correlações

### 7.1 Análise de Correlações com o Churn

In [ ]:
# Preparar dados para correlação
df_corr = df.copy()

# Converter Churn para binário
df_corr['Churn_Binary'] = (df_corr['Churn'] == 'Yes').astype(int)

# Converter categóricas para numéricas (Label Encoding)
from sklearn.preprocessing import LabelEncoder

categorical_cols = df_corr.select_dtypes(include='object').columns.tolist()
categorical_cols.remove('customerID')  # Remover ID
if 'Churn' in categorical_cols:
    categorical_cols.remove('Churn')  # Já temos Churn_Binary

le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_corr[f'{col}_encoded'] = le.fit_transform(df_corr[col])
    le_dict[col] = le

# Selecionar colunas numéricas e encodadas
numeric_cols = df_corr.select_dtypes(include=np.number).columns.tolist()

# Calcular correlação
corr_matrix = df_corr[numeric_cols].corr()

# Visualização - Heatmap completo
plt.figure(figsize=(16, 14))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlação - Todas as Variáveis', fontweight='bold', fontsize=16)
plt.tight_layout()
plt.show()

### 7.2 Correlação com Churn



In [ ]:
# Correlação com Churn
churn_corr = corr_matrix['Churn_Binary'].sort_values(ascending=False)
print("\nCorrelação com Churn (Top 15):\n")
print(churn_corr.head(15))

In [ ]:
# Visualização - Correlação com Churn
plt.figure(figsize=(10, 8))
churn_corr_top = churn_corr[1:16]  # Excluir correlação consigo mesmo
sns.barplot(x=churn_corr_top.values, y=churn_corr_top.index, palette='RdYlGn_r')
plt.title('Top 15 Variáveis Correlacionadas com Churn', fontweight='bold', fontsize=14)
plt.xlabel('Correlação')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

### Insights - Correlações

**Correlações Positivas com Churn (aumentam churn):**
 - Contract Month-to-month
 - Fiber optic internet
 - Electronic check payment
 - Paperless billing
 - Sem serviços de segurança

**Correlações Negativas com Churn (reduzem churn):**
 - Tenure (tempo de contrato)
 - Contract de 1 ou 2 anos
 - TotalCharges
 - Serviços de segurança (OnlineSecurity, TechSupport)

**Conclusão:** Variáveis de contrato e serviços adicionais são os principais fatores.


## 8. Segmentação de Clientes

Criar segmentos de clientes para análise mais granular.

### 8.1 Criar segmentos baseados em tenure

In [ ]:
def categorize_tenure(tenure):
    if tenure <= 12:
        return 'Novo (0-12 meses)'
    elif tenure <= 24:
        return 'Intermediário (13-24 meses)'
    elif tenure <= 48:
        return 'Estabelecido (25-48 meses)'
    else:
        return 'Veterano (48+ meses)'

df['Tenure_Segment'] = df['tenure'].apply(categorize_tenure)

### 8.2 Criar segmentos baseados em MonthlyCharges

In [ ]:
def categorize_charges(charges):
    if charges <= 35:
        return 'Baixo (≤R$35)'
    elif charges <= 70:
        return 'Médio (R$35-70)'
    else:
        return 'Alto (>R$70)'

df['Charges_Segment'] = df['MonthlyCharges'].apply(categorize_charges)

### 8.3 Análise de churn por segmento

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Churn por Tenure Segment
tenure_seg_churn = pd.crosstab(df['Tenure_Segment'], df['Churn'], normalize='index') * 100
tenure_order = ['Novo (0-12 meses)', 'Intermediário (13-24 meses)',
                'Estabelecido (25-48 meses)', 'Veterano (48+ meses)']
tenure_seg_churn = tenure_seg_churn.reindex(tenure_order)
tenure_seg_churn.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'], rot=45)
axes[0].set_title('Taxa de Churn por Segmento de Tenure', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Percentual (%)')
axes[0].set_xlabel('Segmento de Tenure')
axes[0].legend(['Não Churn', 'Churn'])

# Churn por Charges Segment
charges_seg_churn = pd.crosstab(df['Charges_Segment'], df['Churn'], normalize='index') * 100
charges_order = ['Baixo (≤R$35)', 'Médio (R$35-70)', 'Alto (>R$70)']
charges_seg_churn = charges_seg_churn.reindex(charges_order)
charges_seg_churn.plot(kind='bar', ax=axes[1], color=['#2ecc71', '#e74c3c'], rot=45)
axes[1].set_title('Taxa de Churn por Segmento de Cobrança', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Percentual (%)')
axes[1].set_xlabel('Segmento de Cobrança')
axes[1].legend(['Não Churn', 'Churn'])

plt.tight_layout()
plt.show()

### 8.4 Taxa de Churn por Segmento de Tenure

In [ ]:
print("\nTaxa de Churn por Segmento de Tenure:")
display(tenure_seg_churn.round(1))

### 8.5 Taxa de Churn por Segmento de Cobrança:

In [ ]:
print("\nTaxa de Churn por Segmento de Cobrança:")
display(charges_seg_churn.round(1))

### Insights - Segmentação

**Por Tenure:**

 - **Novos clientes (0-12 meses):** ~50% de churn (CRÍTICO)
 - **Intermediários (13-24 meses):** ~35% de churn (ALTO)
 - **Estabelecidos (25-48 meses):** ~15% de churn (MÉDIO)
 - **Veteranos (48+ meses):** ~7% de churn (BAIXO)

**Por Cobrança:**

 - **Baixo (≤ 35):** ~12% de churn
 - **Médio (35-70):** ~25% de churn
 - **Alto (> 70):** ~33% de churn

**Conclusão:** Primeiros 12 meses são **críticos**. Clientes com cobranças altas também são risco.


## 9. Perfil do Cliente em Risco de Churn

Criando um perfil detalhado do cliente típico que faz churn.

In [ ]:
#Separar clientes que fizeram churn
churned = df[df['Churn'] == 'Yes']
not_churned = df[df['Churn'] == 'No']

In [ ]:
#Comparação de perfis
profile_comparison = pd.DataFrame({
    'Característica': [
        'Tenure Médio (meses)',
        'Cobrança Mensal Média (R$)',
        'Cobrança Total Média (R$)',
        '% Senior Citizens',
        '% Sem Parceiro',
        '% Sem Dependentes',
        '% Contrato Mensal',
        '% Fiber Optic',
        '% Sem OnlineSecurity',
        '% Electronic Check'
    ],
    'Churn': [
        churned['tenure'].mean(),
        churned['MonthlyCharges'].mean(),
        churned['TotalCharges'].mean(),
        (churned['SeniorCitizen'] == 1).mean() * 100,
        (churned['Partner'] == 'No').mean() * 100,
        (churned['Dependents'] == 'No').mean() * 100,
        (churned['Contract'] == 'Month-to-month').mean() * 100,
        (churned['InternetService'] == 'Fiber optic').mean() * 100,
        (churned['OnlineSecurity'] == 'No').mean() * 100,
        (churned['PaymentMethod'] == 'Electronic check').mean() * 100
    ],
    'Não Churn': [
        not_churned['tenure'].mean(),
        not_churned['MonthlyCharges'].mean(),
        not_churned['TotalCharges'].mean(),
        (not_churned['SeniorCitizen'] == 1).mean() * 100,
        (not_churned['Partner'] == 'No').mean() * 100,
        (not_churned['Dependents'] == 'No').mean() * 100,
        (not_churned['Contract'] == 'Month-to-month').mean() * 100,
        (not_churned['InternetService'] == 'Fiber optic').mean() * 100,
        (not_churned['OnlineSecurity'] == 'No').mean() * 100,
        (not_churned['PaymentMethod'] == 'Electronic check').mean() * 100
    ]
})

display(profile_comparison.round(2))

In [ ]:
# Visualização
fig, ax = plt.subplots(figsize=(14, 8))

x = np.arange(len(profile_comparison))
width = 0.35

bars1 = ax.barh(x - width/2, profile_comparison['Churn'], width,
                label='Churn', color='#e74c3c', alpha=0.8)
bars2 = ax.barh(x + width/2, profile_comparison['Não Churn'], width,
                label='Não Churn', color='#2ecc71', alpha=0.8)

ax.set_xlabel('Valor', fontweight='bold')
ax.set_title('Comparação de Perfis: Churn vs Não Churn', fontweight='bold', fontsize=16)
ax.set_yticks(x)
ax.set_yticklabels(profile_comparison['Característica'])
ax.legend()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### Perfil Típico do Cliente em Risco

**Cliente com ALTA probabilidade de churn:**

1. **Novo cliente** (tenure < 12 meses)
2. **Cobrança mensal alta** (> R$ 70)
3. **Contrato mensal** (Month-to-month)
4. **Internet Fiber optic**
5. **Sem serviços de segurança** (OnlineSecurity, TechSupport)
6. **Pagamento via Electronic check**
7. **Sem parceiro/dependentes**
8. **Senior citizen**

**Ação prioritária:** Focar retenção neste perfil nos primeiros 6 meses.

## 10. Principais Insights de Negócio

**Consolidação dos insights mais importantes para ação.**

1. **CONTRATO MENSAL É O MAIOR RISCO**

   • Taxa de churn: 42% (vs 11% anual, 3% bianual)

   • Ação: Campanha agressiva para migração de contratos

   • ROI estimado: Redução de 30% no churn total

2. **PRIMEIROS 12 MESES SÃO CRÍTICOS**

   • 50% dos churns ocorrem nos primeiros 12 meses

   • Ação: Programa de onboarding e acompanhamento intensivo

   • Meta: Reduzir churn de novos clientes para 30%

3. **FIBER OPTIC TEM PROBLEMA DE RETENÇÃO**

   • Taxa de churn: 42% (vs 19% DSL)

   • Ação: Investigar qualidade/satisfação e ajustar preço/valor

   • Considerar: Pacotes promocionais ou upgrades de serviço


4. **SERVIÇOS DE SEGURANÇA RETÊM CLIENTES**

   • Clientes com OnlineSecurity/TechSupport têm 50% menos churn

   • Ação: Oferecer 3 meses grátis de serviços de segurança

   • Cross-sell estratégico para aumentar valor percebido


5. **ELECTRONIC CHECK É INDICADOR DE RISCO**

   • Taxa de churn: 45% (vs 15-18% outros métodos)

   • Ação: Incentivar migração para débito automático com desconto

   • Benefício adicional: Redução de inadimplência

## Impactos Financeiros e Próximas Ações:

In [ ]:
print("IMPACTO FINANCEIRO ESTIMADO:\n")

print(f"• Receita mensal em risco: {churned['MonthlyCharges'].sum():,.2f}")
print(f"• Receita anual em risco: {churned['MonthlyCharges'].sum() * 12:,.2f}")
print(f"• Potencial de recuperação (30%): {churned['MonthlyCharges'].sum() * 12 * 0.3:,.2f}\n")

print("PRÓXIMOS PASSOS:\n")

print("1. Desenvolver modelo preditivo (Notebook 03)")
print("2. Criar sistema de recomendação de ações (Notebook 04)")
print("3. Implementar dashboard executivo (Streamlit)")
print("4. Pilotar campanhas de retenção nos segmentos de alto risco")

## 11. Exportação de Dados para Modelagem

Salvando os dados com as novas features criadas.

In [ ]:
# Adicionar features criadas
df_export = df.copy()

# Salvar
df_export.to_csv('/content/churn_data_with_segments.csv', index=False)

# Verificar
import os
print(f"Salvo em: /content/churn_data_with_segments.csv")
print(f"Tamanho: {df_export.shape}")


## 12. Conclusão do EDA
---

**O que descobrimos:**

1. **Dataset balanceado?** Não (73% não churn, 27% churn)
2. **Principal fator de risco?** Contrato mensal (42% churn)
3. **Período crítico?** Primeiros 12 meses (50% churn)
4. **Serviço problemático?** Fiber optic (42% churn)
5. **Fator protetor?** Serviços de segurança e contratos longos

### Variáveis mais importantes identificadas:
---

- Contract (tipo de contrato)
- tenure (tempo de relacionamento)
- InternetService (tipo de internet)
- OnlineSecurity, TechSupport (serviços adicionais)
- PaymentMethod (forma de pagamento)
- MonthlyCharges (valor mensal)

### Próximos Passos:
---

**Notebook 03 - Feature Engineering:**

- Criar features derivadas
- Encoding de variáveis categóricas
- Normalização/padronização
- Preparação final para modelagem

**Notebook 04 - Modeling:**

- Treinar modelos de classificação
- Avaliar performance
- Selecionar melhor modelo
- Interpretar resultados

## Resumo do Notebook 02

---
Este notebook realizou uma análise exploratória completa com:

- Análise univariada de todas as variáveis
- Análise da variável alvo (Churn)
- Análise bivariada (relações com Churn)
- Matriz de correlações
- Segmentação de clientes
- Perfil de risco
- Insights acionáveis de negócio
---

## Principais Descobertas:
---
- Contrato mensal: 42% de churn (maior fator de risco)

- Primeiros 12 meses: 50% de churn (período crítico)

- Fiber optic: 42% de churn (problema de qualidade/valor)

- Serviços de segurança: Reduzem churn em 50%

- Electronic check: 45% de churn (indicador de risco)

---

**Notebook 02 Concluído!**